TODO:
Add tags for sound retrievel (sound, nosound, +/- error?)
Add part of speech parsing/tag
Add check for irregular verbs
    - if irregular, manual review (vs long-term automate creation of irregular past tense cards?)
Synonyms...
Uncouple fetch audio method & translationPair class?
Move more of your main script into functions for a cleaner look. Something like 
    for word in words:
        processWord(word);
    where processWord would fetch the page, parse it, fetch the audio file...etc.

DONE: 
Add retry logic to fetch wiki page
Skip header of transcribed file. 
Add skip audio file fetch if file already exists

In [13]:
from bs4 import BeautifulSoup
import requests, os, time, csv, random

In [14]:
class TranslationPair:
    def __init__(self, dutch_word, english_translation, processed_vocab, wiki_url = None, wiki_page = None, audio_url = None, tags=''):
        self.dutch_word = dutch_word
        self.english_translation = english_translation
        self.processed_vocab = processed_vocab
        self.wiki_url = wiki_url
        self.wiki_page = wiki_page
        self.audio_url = audio_url
        self.tags = tags

In [15]:
def get_wait_time():
    wait_time = random.uniform(1, 2)
    return wait_time

In [16]:
# Read csv file (assumes Column 1 = dutch word/phrase & Colum 2 = English translation)
# remove article if noun; remove preposition if present; replace spaces with _ if multiple words

def process_vocab(vocab_file_path):
    with open(vocab_file_path, newline='', encoding = 'utf-8-sig') as csvfile:
        vocab_list = csv.reader(csvfile, delimiter=',', quotechar='"')
        processed_vocab_list = []
        
        next(vocab_list, None)  # skip the headers

        for word_pair in vocab_list:
            dutch_phrase = word_pair[0]
            english_phrase = word_pair[1]

            # check for article or reflexive & remove if present
            segmented_dutch = dutch_phrase.split(' ')
            if segmented_dutch[0] in ('de', 'het', 'zich'):
                word = segmented_dutch[1]
                #print(f'{word_pair[0]} had an article; final word: {word}')

            # if there is no article...
            else:
                # is the phrase made up of multiple words?
                if len(segmented_dutch) > 1:

                    # check for preposition & remove if present
                    if segmented_dutch[1].startswith('('):
                        word = segmented_dutch[0]
                        #print(f'{word_pair[0]} had a preposition/reflexive; final word:  {word}')

                    # not a preposition but multiple words
                    else:
                        word = word_pair[0].replace(' ', '_') 
                        #print(f'{word_pair[0]} is multiple words; final word:  {word}')

                # no second word found
                else:
                    word = word_pair[0] 
                    #print(f'{word_pair[0]} has no article/preposition; final word:     {word}') 

            # Make list of processed words & their translations
            word_pair = TranslationPair(dutch_phrase, english_phrase, word)
            processed_vocab_list.append(word_pair)
    
    return processed_vocab_list
    
    

In [17]:
# Fetch wiki page for each word in vocab list

def fetch_wiki_page(session, word):
    url = f"http://nl.wiktionary.org/wiki/{word.processed_vocab}"
    print(url)
    response = session.get(url)#, headers=headers)
    attempt = 0
    while attempt < 5:
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            word.wiki_url = url
            word.wiki_page = soup       
            break
        elif response.status_code == 404:
            print(f'No wiki page exists for {word.processed_vocab}.')
            break
        else:
            print(f"Failed to fetch page for {word.processed_vocab} (Status Code: {response.status_code}); attempt = {attempt}")
            time.sleep(30) # retry in 30 seconds
            attempt += 1


In [18]:
# Parse relevant info from wiki page (only sound file for now)
def parse_wiki_page(word):
    page = word.wiki_page
    # parsing audio file from scraped wiki page
    # Find <span> elements with the class "IPA unicode audiolink"
    if page:
        spans = page.find_all('span', class_="IPA unicode audiolink")
        if spans:
            if len(spans) > 1:
                print(f'{word.processed_vocab} has multiple sound files;    downloading only the first. /n Check word listing for other files:     {word.wiki_url}')
            for span in spans:
                # Find the <a> tag inside the <span>
                link = span.find('a', class_="internal")

                if not link and not link['href']:
                    print(f'No href link found for {word}')

                elif link and link['href']:
                    word.audio_url = "http:" + link['href']  # Construct the full   URL                                 
                    break  # Exit after recording the first valid audio file
                else:
                    print(f"No valid audio link found for {word}")
        else:
            print(f"No audio links found for {word}")
    else: print(f'No parsing performed for {word.processed_vocab}.')

In [19]:
# Fetch audio files for each word from wiktionary page
# If multiple audio files are listed, the user is informed & the first is used.

def fetch_audio(session, word, save_folder):
    file_name = f"Nl-{word.processed_vocab}.ogg" # matches default naming   convention for manual file download
    file_path = os.path.join(save_folder, file_name)
    
    if word.audio_url:
        with open(file_path, 'wb') as f:
            attempt = 0
            while attempt < 5:
                audio_response = session.get(word.audio_url)#, headers=headers)
                audio_file = audio_response.content
                if audio_response.status_code == 200:
                    f.write(audio_file)
                    print(f"Downloaded: {file_name} from {word.audio_url}")
                    return file_name # exit after downloading successfully
                else:
                    print(f"Failed to download: {file_name} from {word.audio_url}   dt {audio_response.status_code}; attempt = {attempt}")
                    time.sleep(30) # retry in 30 seconds
                    attempt += 1
    else: print(f'No audio file retrieved for {word.processed_vocab}.')

In [20]:
def write_to_file(write_data, file_location):
    with open(file_location, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for k in write_data.items():
            writer.writerow(k)
    print('File complete.')


In [21]:
# hardcoded test file
vocab_file_path = r"C:\Users\wisery\Downloads\Dutch-English-Vocabulary.csv"

# avoid bot flagging
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
session.headers.update(headers)

# prepare save locations
audio_save_folder = r"C:\Users\wisery\Data Science Projects\Anki Deck\test sound files"
vocab_output_location = r'C:\Users\wisery\Data Science Projects\Anki Deck\audio_added_vocab.csv'

#r'C:\Users\wisery\AppData\Roaming\Anki2\User 1\collection.media' #default Anki media storage location

# r"C:\Users\wisery\Data Science Projects\Anki Deck\test sound files"
# test data storage
os.makedirs(audio_save_folder, exist_ok=True)

existing_sound_files = os.listdir(audio_save_folder) # to compare to prevent re-downloading files

# prepare words       
words = process_vocab(vocab_file_path) # yields list of TranslationPair objects
print('All vocab processed.')

write_data = {}
# Fetch & parse wiki page + fetch audio file for each word in vocab list
for word in words:
    fetch_wiki_page(session, word)
    parse_wiki_page(word)
    if f"Nl-{word.processed_vocab}.ogg" in existing_sound_files:
        audio_file_name = 'Nl-'+ word.processed_vocab +'.ogg'
        print(f'Audio previously downloaded for {word.processed_vocab}. Skipping download...')

    elif f"Nl-{word.processed_vocab}.ogg" not in existing_sound_files:
        time.sleep(get_wait_time())
        audio_file_name = fetch_audio(session, word, audio_save_folder)

    # modify formatting based on if audio file download was successful
    if not audio_file_name:
        dutch_complete = word.dutch_word
    else:
        dutch_complete = f'{word.dutch_word} \n[sound:{str(audio_file_name)}]'

    write_data[dutch_complete]= f'{word.english_translation}'

print('Sound files collected; writing to file.')

write_to_file(write_data, vocab_output_location)

All vocab processed.
http://nl.wiktionary.org/wiki/aan_de_bak_moet
No wiki page exists for aan_de_bak_moet.
No parsing performed for aan_de_bak_moet.
Audio previously downloaded for aan_de_bak_moet. Skipping download...
http://nl.wiktionary.org/wiki/smokkelen
Audio previously downloaded for smokkelen. Skipping download...
http://nl.wiktionary.org/wiki/ongrijpbaar
Audio previously downloaded for ongrijpbaar. Skipping download...
http://nl.wiktionary.org/wiki/levensbeschouwelijk
Audio previously downloaded for levensbeschouwelijk. Skipping download...
http://nl.wiktionary.org/wiki/in_grote_lijnen
No wiki page exists for in_grote_lijnen.
No parsing performed for in_grote_lijnen.
No audio file retrieved for in_grote_lijnen.
http://nl.wiktionary.org/wiki/luguber
Audio previously downloaded for luguber. Skipping download...
http://nl.wiktionary.org/wiki/saamhorigheid
Audio previously downloaded for saamhorigheid. Skipping download...
http://nl.wiktionary.org/wiki/sieren
Audio previously down